# Example deck for running a discontinuity tolerant DIC analysis on a set of images

In [ ]:
import os as o
from pathlib import Path
import time
import matplotlib.pyplot as plt
import crosspy as xpy





## Fill settings below

In [ ]:

# Define the folder path to images
folder_path=Path(r'C:\Alex\crosspy\data\ce4511_2\aligned_stiches')

# NOTE THAT IMAGES SHOULD BE NUMBERED

# Coarse pass settings
coarse = 200 # in pixels
coarse_overlap = 50 # in percentage

# Fine pass settings
fine = 48 # in pixels
overlap = 80 # in percentage


# Heaviside pass settings
hs_size = 48
hs_overlap = 80

# fft filter settings: high pass, high pass width, low pass, low pass width
filter_settings=[4,2,15,8]



## Here we run conventional DIC to compare with later

In [ ]:
if __name__ == "__main__":

    # build the dic class (but don't run it yet):
    Images = xpy.Imset(folder_path,'tif',[0,1])
    roi_1stpass = dict(size_pass = coarse, overlap_percentage = coarse_overlap, xcf_mesh=coarse)
    dic_1stpass=xpy.DIC(cropped,roi_1stpass,filter_settings)
    # run the dic on specified images within the stack, and get displacements:
    dic_1stpass.run_sequential(cores=40)
    dic_1stpass.plot_displacements()

    # correct the images and instantiate a new DIC class

    corrected_images=dic_1stpass.correct(method='polynomial',printing=1)
    #cropped2 = crop_image(corrected_images, 3000, 2000, 6000-1500, 4000-1000)
    roi_2ndpass = dict(size_pass = fine, overlap_percentage = overlap, xcf_mesh=fine)

    dic_2ndpass = xpy.DIC(corrected_images,roi_2ndpass,filter_settings,savingfolder=dic_1stpass.folder)
    print(dic_2ndpass.n_subsets)
    # # run the second pass
    dic_2ndpass.run_sequential(cores=40)#, hs=False)
    dic_2ndpass.plot_displacements()

    dic_2ndpass.calculate_strain()
    dic_2ndpass.plot_strains()

    #dic_2ndpass.save_data()

## Here we run a third pass based on corrected images after first pass - this pass uses the heaviside algo

In [ ]:
if __name__ == "__main__":

    roi_3rdpass = dict(size_pass = hs_size, overlap_percentage = hs_overlap, xcf_mesh=hs_size)

    dic_3rdpass = xpy.DIC(corrected_images,roi_3rdpass,filter_settings,savingfolder=dic_1stpass.folder)

    print(dic_3rdpass.n_subsets)
    # # run the second pass
    dic_3rdpass.run_sequential(cores=40, hs=True, cc_t=0.)
    dic_3rdpass.plot_displacements()